In [1]:
import pandas as pd
from math import pi, sin, cos, exp, tan, acos

In [2]:
 #d = {'a': 1, 'b': 2, 'c': 3}
#ser = pd.Series(data=d, index=['a', 'b', 'c'])
def f(s):
    s['e'] = 5
#ser['d'] = 4
f(ser)
ser

NameError: name 'ser' is not defined

# Dicinoário de Dados
COEFICIENTE_BASAL    => KCB
COEFICIENTE_CULTURA  => KC
COEFICIENTE_EVAPO    => KE
EVAPO_CULTURA        => ETC
TRANSP_POTENCIAL     => TRP
EVAPO                => E
ARMAZENAMENTO_AGUA   => ARM
Capacidade de Campo  => TAR
ARM2                 => ARM2
EVAPO_REFERENCIA     => ET0
EVAPO_REAL           => ETR
TR                   => TR
TA                   => TA
DIAJULIANO           => DIAJULIANO
TARMEDIO             => TAR/2
Indice de Satisfação => ISNA
PLUVIOSIDADE         => P

In [14]:
#Termo Aerodinâmico    
def termoAeroDinamico(params, urx, urn, tx, tn):
    tm   = (tx + tn)/2
    #Pressão de saturação do vapor d'água do ar - Equação de Tetens [kPa]
    es   = (0.6108* exp(17.27*tx/(237.3+tx))+ 0.6108* exp(17.27*tn/(237.3+tn)))/2
    #Pressão real do vapor d'água do ar [kPa]   
    ea   = (urn * 0.6108* exp(17.27*tx/(237.3+tx))+ urx * 0.6108* exp(17.27*tn/(237.3+tn)))/ 200
    #Declividade da curva de pressão de saturação [kPa/oC]
    S    = 4098 * 0.6108* exp(17.27*tm/(237.3+tm))/((237.3 + tm)**2)
    #Pressão atmosférica [kPa]
    Patm = 101.3*((293-0.0065*params.z)/(293))**5.26
    #Calor latente de evaporação [MJ/kg]
    lamb = 0.665E-3*Patm
    
    return S, tm, es, ea, lamb

#Termo Radioativo
def saldoDeRadiacao(params, doy, qg, tx, tn, ea):
    #Radiação solar extraterrestre

    #Correção relativa Terra-Sol
    dr   = 1 + 0.033 * cos(2*pi/365*doy)
    #Declinação Solar 
    decl = 0.409 * sin((2*pi/365*doy)-1.39)
    #Angulo Horário
    ws   = acos(-(params.fi*pi/180)*tan(decl))
    #Radiação Solar Extraterrestre
    Qo   = 37.568*dr*((ws*sin(params.fi*pi/180)*sin(decl))+(cos(params.fi*pi/180)*cos(decl)*sin(ws)))

    #Balanço de radiação
    #Ondas curtas
    
    #Radiação solar para dia de céu sem nebulosidade
    Qso = (0.75 + 2e-5*params.z)*Qo
    Qoc = 0.77 * qg
    
    #Ondas longas
    Qol    = 4.903E-9*((((tx + 273.16)**4)+ ((tn + 273.16)**4))/2)*(0.34-0.14*ea**0.5)*(1.35*(qg/Qso)-0.35)

    return Qoc - Qol

#Transpiração Potencial
def transpiracaoPotencial(params, u2, urn, aETo_PM, t):
    if t < params.L_ini:
        Kcb = params.Kcb_ini
    elif t < params.L_ini + params.L_cres:
        Kcb = params.Kcb_ini+(t-params.L_ini)/params.L_cres*(params.Kcb_mid-params.Kcb_ini)
    elif t < params.L_ini + params.L_cres + params.L-mid:
        Kcb = params.Kcb_mid
    else:
        Kcb = params.Kcb_mid+(t-(params.L_ini+params.L_cres+params.L_mid))/params.L_fim*(params.Kcb_fim-params.Kcb_mid)
    
    h = max((Kcb/params.Kcb_mid)*params.hx,params.h)
    kcx = max(1.2+(0.04*(u2-2)-0.004*(urn-45))*(params.h/3)**0.3,Kcb+0.05)

    return Kcb, kcx 

#Balanço de água no solo
def evaporacao(params, p, Kcb, kcx, ETo_PM, De_f, t):
    #Camada subsuperfícial - Evaporação
    fc = max(abs(((Kcb-params.Kcb_ini)/(kcx-params.Kcb_ini)))**(1 + 0.5*params.h),0.01)
    
    if p > 0: fw = 1
    else: fw = params.fw_ini
    
    few = min(1-fc,fw)
    TAE = 1000*(params.cc-0.5*params.PM)*params.Ze
    
    if t == 1: De_i = TAE - params.AFE
    else: De_i = max(De_f-p, 0)
        
    if De_i < params.AFE: kr = 1
    else: kr = max((TAE - De_i)/(TAE - params.AFE), 0)
            
    ke = min(kr*(kcx-Kcb), few*kcx)
    E = ke * ETo_PM
    
    if t == 1:
        Dp = max(p, 0)
        De_f = min(De_i - p+(E/few) + Dp, TAE)
    else:
        Dp = max(p-De_f, 0)
        De_f = min(De_f-p+(E/few) + Dp, TAE)
        
    Kc = Kcb + ke
    ETc = Kc * ETo_PM
    return ETc, ke, De_f, De_i


def evapotranspiracaoReal(params, ap, aKcb, aETc, aKe, aETo_PM, De_i, t):
    if t <= (params.L_ini + params.L_cres + params.L_mid):
        Zr = ((aKcb - params.Kcb_ini)/(params.Kcb_mid - params.Kcb_ini))*(params.Zrx - params.Zrn) + params.Zrn
    else:
        Zr = params.Zrx
    
    TAR = 1000.*(params.cc - params.PM)*Zr
    AFR = params.f*TAR
    
    if t == 1:
        Dr_i = min( max((TAR-AFR)-ap+aETc, 0), TAR)
        if Dr_i < AFR:
            ks = 1
        else:
            ks = max( (TAR-Dr_i)/(TAR-AFR), 0)

        Dp = max( ap-aETc-De_i, 0 )
        kcr = aKe + ks * aKcb
        ETR = kcr * aETo_PM
        Tr = ks * aKcb *aETo_PM
        Dr_f = max( min((TAR-AFR)-ap+ETR+Dp, TAR), 0 )
    else:
        Dr_i = min( max(Dr_f-ap-aETc, 0), TAR)
        if Dr_i < AFR:
            ks = 1
        else:
            ks = max( (TAR-Dr_i)/(TAR-AFR), 0)
        
        Dp = max( ap-aETc-Dr_f, 0)
        kcr = aKe + ks*aKcb
        ETR = kcr * aETo_PM
        Tr = ks*aKcb*aETo_PM
        Dr_f = min(Dr_f-ap+ETR+Dp, TAR)
    return  TAR, Dr_f

In [8]:
#Precisa instalar xlrd ("pip install xlrd" ou "conda install xlrd")
dados = pd.read_excel("datasets/DadosFeijao_Chuva.xls")
params = pd.read_excel("datasets/ParametrosFeijao.xls").squeeze()

In [15]:
aARM = []
De_f = 0
De_i = 0
for idx, (doy, tx, tn, urx, urn, u2, qg, ap, aArm, teta, chuva, irriga, _) in dados.iterrows():
    
    # Termo Aerodinâmico    
    S, tm, es, ea, lamb = termoAeroDinamico(params, tx, tn, urn, urx)
    
    Qn = saldoDeRadiacao(params, doy, qg, tx, tn, ea)
    
    #Evapotranspiração de referência [mm/d]
    aETo_PM = (0.408*S*(Qn)+lamb*900*u2*(es-ea)/(tm + 273))/(S+lamb*(1+0.34*u2))
    
    aKcb, kcx = transpiracaoPotencial(params, u2, urn, aETo_PM, idx+1)
    aETc, aKe, De_f, De_i = evaporacao(params, ap, aKcb, kcx, aETo_PM, De_f, idx+1)
    aTAR, Dr_f = evapotranspiracaoReal(params, ap, aKcb, aETc, aKe, aETo_PM, De_i,idx+1)
    
    aARM.append( max(aTAR - Dr_f,0) )
    break

In [33]:
dados.columns, params.index

(Index(['DOY', 'TX', 'TN', 'URX', 'URN', 'U2', 'QG', 'P', 'ARM', 'TETA', 'P.1',
        'I', 'EXP'],
       dtype='object'),
 Index(['z', 'fi', 'L_ini', 'L_mid', 'L_cres', 'L_fim', 'Kcb_ini', 'Kcb_mid',
        'Kcb_fim', 'hx', 'h', 'fw_ini', 'cc', 'PM', 'Ze', 'Zrn', 'Zrx', 'AFE',
        'f', 't'],
       dtype='object'))

In [132]:
De_f

17.325000000000003